# Info 370 HW 03: Webscraping

Name: Ella Kim

##  Legal and Ethical Issues
### 1 

Find a website that you consider scraping:

Main URL: 
[What's in the pan](https://whatsinthepan.com/)  

### 2

Discuss the legality of scraping this site: does it mention data collection in its terms-ofservice? Does robots.txt file allow/disallow what you want to do?

Ans: There did not seem to be a terms of use page, and only seems to state that they are an affiliate program with Amazon Services LLC Associates Program. I confirm with the directions page that they do not seem to have anything against scraping data. The robots.txt file only has that there is no access to admins but to admin-ajax. 

[Citing of statments above](https://whatsinthepan.com/robots.txt)

### 3
Discuss what kind of steps do you take to lessen the burden to the provider. Consider:  

(a) Only downloading minimum amount necessary for developing and debugging your code, and
dowloading the full amount just once. Note that this PS asks you to scrape 50 recipies, no
more, no less. So you may stay with downloading only 60, just in case some of those are too
complex to handle.  
(b) Adding a wait time (e.g. 1s) between successive page downloads  
(c) In case of wikimedia, I donated them money as a “thank you” for the open access.  

Ans: I will go with the suggestions in the directions and only scrape 60 recicpies, and attempt to add a 1s wait time between each page's download (as too fast of navigation could be considered spam, etc.)




## Scrape the recipies

### 1

Write the code to scrape (at least 50) recipes from the webpage you chose.

[Citing .get for url](https://www.kite.com/python/answers/how-to-get-href-links-from-urllib-urlopen-in-python)  

In [1]:
# IMPORTS
from bs4 import BeautifulSoup
from urllib.request import urljoin
import requests
import pandas as pd
import numpy as np
from fractions import Fraction
import time

# function to get a page: to remove repetition in code
def getPage(url):
    response  = requests.get(url)
    # comment and uncomment to make sure got page successfully for each one
    # print("Status:", response.status_code)
    webPage = response.content
    soup = BeautifulSoup(webPage, "html.parser")
    type(soup)
    return(soup)

# first main page
soupStart = getPage("https://whatsinthepan.com/recipes/")

# find the pages with different recipies
findCategories = soupStart.body.find_all("p", class_ = "more-from-category")

# function to find links to main catagory pages from subtexts, to remove code repetition
def getLinks(rawLink):
    currList = []
    # loop through each subtext with a url to find
    for index in range(0, len(rawLink)):
        more = rawLink[index] # get subtext
        link = more.find("a") # get a bc that is the start of a url
        currList.append(link.get('href')) # get the url
    return(currList)

# check if got links to catagories correctly 
linksToCat = getLinks(findCategories)
linksToCat # sanity check

# make empty list for all recipe links
allRecipes = []
# loop to find all recipe page links
for index in range(0, len(linksToCat)):
    soupCurrCategory = getPage(linksToCat[index])
    time.sleep(1) # to be nice to the website
    # the specific type and class for the different categories of recipes
    findSection = soupCurrCategory.body.find_all("h2", class_ = "entry-title")
    linksRecipes = getLinks(findSection) # use function from above 
    allRecipes.extend(linksRecipes)

allRecipes #sanity check

# prove scraped recipies form webpage I chose
print("Scraped the url for all unique recipies on site, total: " + str(len(allRecipes)))


Scraped the url for all unique recipies on site, total: 50


### 2

Pull out the list of ingredients, and split those into quantities, units, and ingredients as well as
you can.  

Hint: sometimes you find unstructured ingredient lines. In that case there is little you can do
with it. It is extremely hard to tell what is what in “a handful of sugar peas” or how much exactly
is “a bunch” of finely chopped cilantro

[Cite for addToList function](https://stackoverflow.com/questions/50661994/beautifulsoup-return-none-if-html-element-not-found)


In [7]:
# make empty lists for all columns for data set
id_num = []
name = []
text = []
quantity = []
unit = []
ingredient = []

# function to add values to column lists, returns the valid value if there is such a value
# for that ingredient, returns N/A if it does not exist for that ingredient 
def addToList(currVal):
    if currVal:
        return(currVal.text)
    else:
        return("N/A")

# function to get a title for each of the recipes 
# (reasoning: the titles for some of the recipes were not literal tiles, the url did it less) 
# returns the part of the url as a "nice"-looking name
def getName(string):
    cutUrl = string.split('/')
    rightIndex = cutUrl[-2] #can only do this because all of urls have same base pattern
    return(rightIndex.replace("-", " "))
    
# to find all recipes, each loop is the different recipes pages (so 50x)
for index in range(0, len(allRecipes)):
    # trying to be nice to website (section 1 about Legal and Ethical Issues...)
    # makes this section of the code take a long time!!
    time.sleep(1)
    # using function made in previous section to get the url page
    soupCurrCategory = getPage(allRecipes[index])
    # using function made above to get the name of the recipe, making constants for each recipe
    curr_name = getName(allRecipes[index])
    curr_id = index + 1 # python indexing starting at 0...
    findIngredientSection = soupCurrCategory.body.find_all("div", class_ = "wprm-recipe-ingredient-group")
    # to find all the different sections of ingredients in one recipe (differs for each page/recipe)
    for index in range(0, len(findIngredientSection)):
        currIngrSection = findIngredientSection[index]
        currList = currIngrSection.find_all("li", class_ = "wprm-recipe-ingredient")
        # to find each ingredient and add each part of the ingredient to the column lists for df
        for index in range(0, len(currList)):
            ingredient_url = currList[index]
            # constants for each separate recipe
            id_num.append(curr_id)
            name.append(curr_name)
            # adding to quality list
            currQuantity = addToList(ingredient_url.find("span", class_ = "wprm-recipe-ingredient-amount"))
            quantity.append(currQuantity)
            # adding to unit list
            currUnit = addToList(ingredient_url.find("span", class_ = "wprm-recipe-ingredient-unit"))
            unit.append(currUnit)
            # adding to ingredient list
            currIngredient = addToList(ingredient_url.find("span", class_ = "wprm-recipe-ingredient-name"))
            ingredient.append(currIngredient)
            # adding to text list
            faded = addToList(ingredient_url.find("span", class_ = "wprm-recipe-ingredient-notes wprm-recipe-ingredient-notes-faded"))
            detailed = [currQuantity, currUnit, currIngredient, faded]
            # to add all the different part before for text column in df
            textRough = ' '.join(detailed)
            text.append(textRough.replace('N/A', ""))

# to check lists all have same length
print("Length of all lists (sanity check): " + str(len(id_num)) + " " + str(len(name)) + " " + str(len(text)) + " " + str(len(unit)) + " " + str(len(quantity)) + " " + str(len(ingredient)))

Length of all lists (sanity check): 582 582 582 582 582 582


Did not print lists:  ingredients are split into their own lists, will put into df in next part of question  

**Because of wait time, this section of code takes a long time to run**

### 3

Store the ingredients from each recipe in a data frame. The data frame should contain the
following variables:  

**id** recipe id (just a number)  
**name** recipe name  
**text** the ingredient line as text as printed on the webpage, stripped of all html attributes  
**quantity** quantity of the ingredient  
**unit** unit of the quantity  
**ingredient** the name/explanation of the ingredient  

The data frame should be in long form, i.e. one line for each ingredient, and several lines for each
recipe.  

Explain what are the issues you cannot easily solve

##### A)

In [17]:
# seems tedious, but first line below did not work because lists were in columns not row dimentions

#df_ingredients = pd.DataFrame([id_num, name, text, quantity, unit, ingredient], columns = ["id", "name", "text", "quantity", "unit", "ingredient"])

# to create a df and add all the lists made in the previous section into a data frame of all ingredients for all recipes
df_ingredients = pd.DataFrame({'id': id_num})
df_ingredients['name'] = name
df_ingredients['text'] = text
df_ingredients['quantity'] = quantity
df_ingredients['unit'] = unit
df_ingredients['ingredient'] = ingredient

# print a preview of data frame as sanity check
print("Preview of data frame: \n")
print(df_ingredients)

Preview of data frame: 

     id                                             name  \
0     1  roasted rolled turkey breast garlic herb butter   
1     1  roasted rolled turkey breast garlic herb butter   
2     1  roasted rolled turkey breast garlic herb butter   
3     1  roasted rolled turkey breast garlic herb butter   
4     1  roasted rolled turkey breast garlic herb butter   
..   ..                                              ...   
577  50                                 buckwheat crepes   
578  50                                 buckwheat crepes   
579  50                                 buckwheat crepes   
580  50                                 buckwheat crepes   
581  50                                 buckwheat crepes   

                                                  text quantity    unit  \
0                     5 tbsp butter softened (divided)        5    tbsp   
1                               6 cloves garlic minced        6  cloves   
2    2 tbsp fresh herbs (thym

##### B)
Some issues I could not solve in the beginning was when there was no quantity, but I then I made a function that would check whethere tehre was no value and would replace the placement with an N/A (so not all values had quantities or details like "(melted)" so those were removed from the text to look nicer)

### 4

Store your dataframe in a file so you don’t have to repeat scraping.

In [11]:
# saving csv with title "ingredients"
df_ingredients.to_csv('ingredients.csv')
df_getdf = pd.read_csv('/home/jovyan/lost+found/INFO_370/ingredients.csv')
df_getdf # sanity check

# remove first column, preventing from weird formatting while working with df here (not sure why it does that when it saves)
df_recipes = df_getdf.iloc[: , 1:]

## What is more common: sweet or salty food?
### 1

For each recipe, find if it contains salt, and if it contains sugar. Are there more recipes with salt
or more with sugar?  

Hint: check out functions: pd.Series.str.contains and np.any.


In [13]:
# filer csv for only salt and sugar into series
ser_ingredients = df_recipes.ingredient
ser_has_salt = ser_ingredients.str.contains(pat = 'salt')
ser_has_sugar = ser_ingredients.str.contains(pat = 'sugar')

# count number of values (i.e. number of times salt is used) & print results
count_salt = sum(ser_has_salt.values.tolist())
print("Recipes with salt: " + str(count_salt))

# count number of values (i.e. number of times sugar is used) & print results
count_sugar = sum(ser_has_sugar.values.tolist())
print("Recipes with sugar: " + str(count_sugar))

# print final answer
print("\nComparing these two values, there are more recipies with salt")

Recipes with salt: 49
Recipes with sugar: 21

Comparing these two values, there are more recipies with salt


### 2

But what about the absolute quantities? Do all these recipes contain more salt or more sugar?
Add the salt and sugar quantities over all recipes and see which one wins.  

If you cannot do this, explain what are the problems!  

**Some of the values are just a sprinkle on top, etc. (i.e. they did not have a concrete value, so they were not included in the final calculations)**

In [19]:
## SALT calculations:

# rough draft of salt df
df_salt_rough = df_recipes[df_recipes['ingredient'].str.contains(pat = "salt")]
df_salt_rough # sanity check

# after inspection, there were some ingredients that had unsalted, so removed those
df_salt_rough_1 = df_salt_rough[~df_salt_rough['ingredient'].str.contains(pat = "unsalted")]

# remove N/A values for quantity 
df_salt = df_salt_rough_1[~df_salt_rough_1.quantity.isnull()]

# see what unique measuring units there are: salt has tsp and tbsp in abbrev and whole word form
unique_salt = df_salt['unit'].unique()
unique_salt # sanity check

# filter separate units to make easier to calculate 
df_salt_tsp = df_salt[df_salt['unit'].isin(unique_salt[0:3])] # has fractions
df_salt_tbsp = df_salt[df_salt['unit'] == (unique_salt[-1])]
df_salt_tsp # sanity check: has weird symbol, replaced below 

# change type object to string
convert_salt_tsp = df_salt_tsp['quantity'].astype('string')
convert_salt_tbsp = df_salt_tbsp['quantity'].astype('string')

# change tsp to numerical (as it does not need further changes from fractions)
salt_tbsp_num = [int(x) for x in convert_salt_tbsp]

# change the weird fraction symbols to actual string versions of fraction
convert_salt_tsp[10] = "1/2"
convert_salt_tsp[499] = "1/4"
convert_salt_tsp # sanity check

# convert series to list
convert_salt_tsp = convert_salt_tsp.tolist()

# make function for conversions of fractions to decimal values
def convertToImproper(lis):
    # new list for converted fractions 
    to_num = []
    # loops through each quantity in list
    for value in lis:
        x = value # itialize to change to numerical & add to new list
        if " " in value: # checking for mixed fraction form 
            x = value.split(" ") # get whole # and fraction
            whole_num = int(x[0])
            frac = Fraction(x[1]) # convert str to fraction
            x = float(frac) + (whole_num) # get final float type
        else: # improper fraction
            x = Fraction(value) #change the type of x from string to Fraction
            x = float(x) #change the type of x from Fraction to float
        to_num.append(x)
    return(to_num)

# use function above for tsp
salt_tsp_num = convertToImproper(convert_salt_tsp)

# get sums of tsp and tbsp quantities (and convert all units to tsp)
sum_salt_tsp = sum(salt_tsp_num)
sum_salt_tbsp_to_tsp = sum(salt_tbsp_num) * 3

# get total in teaspoons & print results
total_salt_tsp = sum_salt_tsp + sum_salt_tbsp_to_tsp
print("Total salt in all recipes (in teaspoons): " + str(total_salt_tsp))

# SUGAR calculations:

# repeat like salt but for sugar:
# rough draft of sugar df
df_sugar_rough = df_recipes[df_recipes['ingredient'].str.contains(pat = "sugar")]
df_sugar_rough # sanity check

# remove N/A values for quantity column
df_sugar = df_sugar_rough[~df_sugar_rough.quantity.isnull()]

#see what unique measuring units there are: sugar has tsp, tbsp, as well as cups, written differently 
unique_sugar = df_sugar['unit'].unique()
unique_sugar.sort() # sorting in alphabetical sorts them in types easily 
unique_sugar # sanity check

# filter separate units to make easier to calculate 
df_sugar_cup = df_sugar[df_sugar['unit'].isin(unique_sugar[0:2])] # has fractions, also has weird element, replace later below
df_sugar_tbsp = df_sugar[df_sugar['unit'].isin(unique_sugar[2:5])] 
df_sugar_tsp = df_sugar[df_sugar['unit'].isin(unique_sugar[5:len(unique_sugar)])]

# change type object to string in series
convert_sugar_cup = df_sugar_cup['quantity'].astype('string')
convert_sugar_tsp = df_sugar_tbsp['quantity'].astype('string')
convert_sugar_tbsp = df_sugar_tsp['quantity'].astype('string')

# change tsp and tbsp to numerical (as it does not need further changes from fractions)
sugar_tsp_num = [int(x) for x in convert_sugar_tsp]
sugar_tbsp_num = [int(x) for x in convert_sugar_tbsp]

# change the weird fraction symbols to actual string versions of fraction
convert_sugar_cup[501] = "1/3"

# convert series to list
convert_sugar_cup = convert_sugar_cup.tolist()

# use function from above in salt section to convert fractions to decimal/float values 
sugar_cup_num = convertToImproper(convert_sugar_cup)

# get sums of tsp, tbsp, and cup lists (and convert all units to tsp)
sum_sugar_cup_to_tsp = sum(sugar_cup_num) * 48
sum_sugar_tsp = sum(sugar_tsp_num)
sum_sugar_tbsp_to_tsp = sum(sugar_tbsp_num) * 3

# get total in teaspoons & print results
total_sugar_tsp = sum_sugar_tsp + sum_sugar_tbsp_to_tsp + sum_sugar_cup_to_tsp
print("Total sugar in all recipes (in teaspoons): " + str(total_sugar_tsp))
print("\nComparing the two, although salt is used more often, it is used in way less of quantities than sugar")

Total salt in all recipes (in teaspoons): 42.5
Total sugar in all recipes (in teaspoons): 568.0

Comparing the two, although salt is used more often, it is used in way less of quantities than sugar


## Finally...

I treated myself to a Korean ramen that a lot of people like to call "fire noodle eating challenge ramen," because spicy food helps me relieve stress.

Duration: ~8 hours
